In [39]:
!pip install nltk
!pip3 install tqdm


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [40]:
import os
import pandas as pd
import numpy as np
import nltk
import pickle
import tensorflow as tf
from os import path
import gensim
from tqdm import tqdm
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras import preprocessing
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout, Embedding, LSTM, add
from keras.utils import to_categorical

nltk.download("stopwords")
stop_words = set(stopwords.words('english'))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


In [42]:
# comment it out if working on local machine. Only required when using google colab
# from google.colab import drive
# drive.mount('/content/drive')

In [43]:
CAPTION_FILE_PATH = "captions.txt"
IMAGE_FOLDER_PATH = "Images"
TRAIN_IMAGE_PICKLE = "train_image_pickle.p"
TEST_IMAGE_PICKLE = "test_image_pickle.p"
W2V_PICKLE_PATH = "w2v_embeddings.p"


In [44]:
df = pd.read_csv(CAPTION_FILE_PATH)

In [45]:
images = df.iloc[:1000, 0]
captions = df.iloc[:1000, 1]
print(images.shape)
print(captions.shape)

(1000,)
(1000,)


In [46]:
print("images>>>>")
print(images[0:10])
print("captions>>>>>")
print(captions[0:10])

images>>>>
0    1000268201_693b08cb0e.jpg
1    1000268201_693b08cb0e.jpg
2    1000268201_693b08cb0e.jpg
3    1000268201_693b08cb0e.jpg
4    1000268201_693b08cb0e.jpg
5    1001773457_577c3a7d70.jpg
6    1001773457_577c3a7d70.jpg
7    1001773457_577c3a7d70.jpg
8    1001773457_577c3a7d70.jpg
9    1001773457_577c3a7d70.jpg
Name: image, dtype: object
captions>>>>>
0    A child in a pink dress is climbing up a set o...
1                A girl going into a wooden building .
2     A little girl climbing into a wooden playhouse .
3    A little girl climbing the stairs to her playh...
4    A little girl in a pink dress going into a woo...
5           A black dog and a spotted dog are fighting
6    A black dog and a tri-colored dog playing with...
7    A black dog and a white dog with brown spots a...
8    Two dogs of different breeds looking at each o...
9      Two dogs on pavement moving toward each other .
Name: caption, dtype: object


In [47]:
#### preprocess the captions #######
for i, caption in enumerate(captions):
    _caption = caption.lower()
    tokens = _caption.split()
    tokens = [word for word in tokens if not word in stop_words]
    # token length greater than one. removes dangling characters
    tokens = [word for word in tokens if len(word)>1]
    # remove tokens with numbers in them
    tokens = [word for word in tokens if word.isalpha()]
    #Add start and end sequence tokens
    captions[i] = "<start> " + " ".join(tokens) + " <end>"


print(captions[0:5])

0    <start> child pink dress climbing set stairs e...
1             <start> girl going wooden building <end>
2    <start> little girl climbing wooden playhouse ...
3    <start> little girl climbing stairs playhouse ...
4    <start> little girl pink dress going wooden ca...
Name: caption, dtype: object


In [48]:
train_x, test_x, train_y, test_y = train_test_split(images, captions)
print(train_x[0:5])
print(train_y[0:5])

581    1122944218_8eb3607403.jpg
523     111537222_07e56d5a30.jpg
792    1176580356_9810d877bf.jpg
911    1224851143_33bcdd299c.jpg
901    1220401002_3f44b1f3f7.jpg
Name: image, dtype: object
581    <start> baby holding small black flag moon sta...
523    <start> woman purple snakeskin pants climbs ro...
792         <start> brown black dog jumps red ball <end>
911           <start> little boy holds spoon mouth <end>
901        <start> two girls laughing outside yard <end>
Name: caption, dtype: object


In [49]:
#### Create a vocabulary from the train captions in train_y #####
#Eliminate infrequently occurring words from the vocabulary

vocabulary = set()
word_counts = {}

for caption in train_y:
    tokens = caption.split(" ")
    for token in tokens:
        vocabulary.add(token)
        if token in word_counts:
            word_counts[token] += 1
        else:
            word_counts[token] = 1

core_vocab = [key for key in word_counts if word_counts[key] >= 5]

In [50]:
#### create a list of captions corresponding to a particular image ######


def create_image_to_caption_dict(images,captions):
  image_to_caption = defaultdict(list)
  for img_, caption in zip(images, captions):
    _image = img_.replace(".jpg", "").strip()
    image_to_caption[_image].append(caption)
  return image_to_caption

train_image_to_caption = create_image_to_caption_dict(train_x, train_y)
test_image_to_caption = create_image_to_caption_dict(test_y, test_y)




In [51]:
model = InceptionV3(weights="imagenet")
model = Model(model.input, model.layers[-2].output)

In [52]:
# Convert all the images to size 299x299 as expected by the

def encode_images(x):
  encoded_images = {}
  
  for _x in tqdm(x):
    image_path = path.join(IMAGE_FOLDER_PATH, _x)
    img = image.load_img(image_path, target_size=(299, 299))
    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    X = preprocess_input(X)
    features = model.predict(X)
    X = np.reshape(features, features.shape[1])
    encoded_images[_x] = X
  return encoded_images




In [53]:
train_encoded_images = {}
test_encoded_images = {}

if os.path.isfile(TRAIN_IMAGE_PICKLE):
    with open(TRAIN_IMAGE_PICKLE, 'rb') as p:
        train_encoded_images = pickle.load(p)
else:
    train_encoded_images = encode_images(set(train_x))
    pickle.dump(train_encoded_images, open("train_image_pickle.p", "wb"))

if os.path.isfile(TEST_IMAGE_PICKLE):
    with open(TEST_IMAGE_PICKLE, 'rb') as p:
        test_encoded_images = pickle.load(p)
else:
    test_encoded_images = encode_images(set(test_x))
    pickle.dump(train_encoded_images, open("test_image_pickle.p", "wb"))




In [54]:
#Create python dictionaries to encode indices of unique words in vocabulary
w2idx = {core_vocab[i]: i for i in range(len(core_vocab))}
idx2w = {i: core_vocab[i] for i in range(len(core_vocab))}

In [55]:
#Get max caption length
max_caption_length = max(len(i.split(' ')) for i in train_y)
print(max_caption_length)

20


In [56]:
#Create dictionary of word embeddings for full vocabulary
# w2v_model = KeyedVectors.load_word2vec_format("glove.6B.200d.txt", binary=False)
# embedding_dict = {i: w2v_model[i] for i in core_vocab if i in w2v_model}
# pickle.dump(embedding_dict, open("w2v_embeddings.p", "wb"))

embedding_dict = {}
with open(W2V_PICKLE_PATH, 'rb') as p:
  embedding_dict = pickle.load(p)

In [57]:
#Create matrix of word embeddings
emb_dimensions = 200

word_embeddings = np.zeros((len(core_vocab), emb_dimensions))

for word, index in w2idx.items():
    if word in embedding_dict:
        word_embeddings[index] = embedding_dict[word]


In [58]:
#Create merge architecture model
image_inputs = Input(shape=(2048,), name="ImageLayer")
image_layer1 = Dropout(0.2)(image_inputs)
image_layer2 = Dense(128, activation = 'relu')(image_layer1)

caption_inputs = Input(shape = (max_caption_length,), name="CaptionLayer")
caption_layer1 = Embedding(len(core_vocab), emb_dimensions, mask_zero = True)(caption_inputs)
caption_layer2 = Dropout(0.5)(caption_layer1)
caption_layer3 = LSTM(128)(caption_layer2)

decoder_layer1 = add([image_layer2, caption_layer3])
decoder_layer2 = Dense(128, activation = 'relu')(decoder_layer1)
output = Dense(len(core_vocab), activation = 'softmax')(decoder_layer2)

model2 = Model(inputs = [image_inputs, caption_inputs], outputs = output)
model2.layers[2].set_weights([word_embeddings])
model2.layers[2].trainable = False

model2.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [59]:
print(model2.summary())


Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
CaptionLayer (InputLayer)       [(None, 20)]         0                                            
__________________________________________________________________________________________________
ImageLayer (InputLayer)         [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 200)      36800       CaptionLayer[0][0]               
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 2048)         0           ImageLayer[0][0]                 
_______________________________________________________________________________________

In [60]:
def generate_data(caption_dict, images, w2idx, max_len, batch_size):
    image_sequence, input_words, output_words = [], [], []
    n = 0
    while True:
        for img_name, caption_list in caption_dict.items():
            n += 1
            photo = images[img_name + ".jpg"]
            for c in caption_list:
              # encode the words in caption using word to index dictionary  
              encoded_caption = [w2idx[w] for w in c.split(" ") if w in w2idx]

              for i in range(1, len(encoded_caption)):
                # input sequence should be upto an index, output should be the index
                input_sequence, output_sequence = encoded_caption[:i], encoded_caption[i]
                # pad the input sequence to the same number of tokens
                input_sequence = pad_sequences([input_sequence], maxlen= max_len)[0]

                # encode the ouput sequence as categorical class
                output_sequence = to_categorical([output_sequence], num_classes=len(core_vocab))[0]

                image_sequence.append(photo)
                input_words.append(input_sequence)
                output_words.append(output_sequence)

            if n == batch_size:
              yield([np.array(image_sequence), np.array(input_words)], np.array(output_words))
              image_sequence, input_words, output_words = [], [], []
              n = 0

In [62]:
num_epochs = 10
batch_size = 5
num_steps = len(train_x) // batch_size
# print(num_steps)
for i in range(num_epochs):
    print("---------------------")
    print("epoch", i)
    data_generator = generate_data(train_image_to_caption, train_encoded_images, w2idx, max_caption_length, batch_size)
    model2.fit(data_generator, epochs = 1, steps_per_epoch = num_steps)
    model2.save('model_epoch_' + str(i) + '.h5')

---------------------
epoch 0
150/150 [==============================] - 5s 35ms/step - loss: 2.2501
---------------------
epoch 1
150/150 [==============================] - 5s 35ms/step - loss: 1.9140
---------------------
epoch 2
150/150 [==============================] - 7s 50ms/step - loss: 1.7398
---------------------
epoch 3
150/150 [==============================] - 8s 55ms/step - loss: 1.5855
---------------------
epoch 4
150/150 [==============================] - 7s 44ms/step - loss: 1.4044
---------------------
epoch 5
150/150 [==============================] - 5s 34ms/step - loss: 1.2692
---------------------
epoch 6
150/150 [==============================] - 5s 34ms/step - loss: 1.1457
---------------------
epoch 7
150/150 [==============================] - 5s 35ms/step - loss: 1.0726
---------------------
epoch 8
150/150 [==============================] - 6s 40ms/step - loss: 0.9414
---------------------
epoch 9
150/150 [==============================] - 5s 36ms/step - los

In [261]:
def generate_caption(image, max_len):
    caption_ = recursive_caption(image, '<start>', max_len)
    caption_list = caption_.split(' ')[1:-1]
    return ' '.join(caption_list)

def recursive_caption(image, current_caption, max_len):
    caption_indices = [w2idx[i] for i in current_caption.split() if i in w2idx]
    padded_caption = pad_sequences([caption_indices], maxlen = max_len + len(caption_indices))

    new_token = idx2w[np.argmax(merge_model.predict([image, padded_caption], verbose=0))]
    caption = current_caption + new_token

    if new_token == '<end>':
        return caption
    if depth == 0:
        return caption
        
    recursive_caption(image, caption, max_len-1)


In [262]:
def model_evaluation(model, caption_list, images, max_len):
    ground_truth, predictions = [], []
    
    for key, captions in caption_list.items():

        prediction = generate_caption(images[key], max_len)
        predictions.append(prediction)

        g_truth = [caption.split() for caption in captions]
        ground_truth.append(g_truth.split())
    
    bleu_1 = corpus_bleu(ground_truth, predictions, weights = [1.0, 0, 0, 0])
    bleu_2 = corpus_bleu(ground_truth, predictions, weights = [.5, .5, 0, 0])
    bleu_3 = corpus_bleu(ground_truth, predictions, weights = [.3, .3, .3, 0])
    bleu_4 = corpus_bleu(ground_truth, predictions, weights = [.25, .25, .25, .25])
    
    return [bleu_1, bleu_2, bleu_3, bleu_4]           